# Black Friday III

Autor: Diego López

Ultima Actualización: 2016-11-27

AnalyticsVhidya link: [practice-problem-big-mart-sales-iii](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/)

El objetivo de este concurso es predecir la demanda de un item en un outlet determinado basado en histórico de ventas y caracteristicas descriptivas del item y el outlet.

In [50]:
import pandas as pd
import numpy as np
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [68]:
full = pd.concat([train, test], ignore_index=True)

In [41]:
print(train.shape)
print(test.shape)
print(full.shape)
full.head()

(550068, 12)
(233599, 11)
(783667, 17)


,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID,AmountBy_User,AmountBy_Product,Product_Category_1_Amount,Product_Category_2_Amount,Product_Category_3_Amount
0,1,0,0,0,10,3,0.0,0.0,684,8370.0,2,0,334093.0,2694686.0,119859.0,0.0,0.0
1,1,0,0,0,10,1,6.0,14.0,2406,15200.0,2,0,334093.0,9472642.0,61604.0,31822.0,27268.0
2,1,0,0,0,10,12,0.0,0.0,868,1422.0,2,0,334093.0,126265.0,61279.0,0.0,0.0
3,1,0,0,0,10,12,14.0,0.0,844,1057.0,2,0,334093.0,496203.0,61279.0,27268.0,0.0
4,8,2,1,0,16,8,0.0,0.0,2769,7969.0,4,1,810472.0,1561631.0,460149.0,0.0,0.0


In [53]:
# Monto y Ranking por Usuario
Amount_byUser = pd.DataFrame(full.groupby(['User_ID'])['Purchase'].sum())
Amount_byUser.columns = ['User_Amount']
Amount_byUser.reset_index(level=0, inplace=True)
Amount_byUser['User_Rank'] = Amount_byUser['User_Amount'].rank(ascending=False)

# Montos por Categoria - Usuario
Amount_CU1 = pd.DataFrame(full.groupby(['User_ID','Product_Category_1'])['Purchase'].sum())
Amount_CU1.reset_index(level=['User_ID','Product_Category_1'], inplace=True)
Amount_CU1.columns = ['User_ID','Product_Category','Amount1']

Amount_CU2 = pd.DataFrame(full.groupby(['User_ID','Product_Category_2'])['Purchase'].sum())
Amount_CU2.reset_index(level=['User_ID','Product_Category_2'], inplace=True)
Amount_CU2.columns = ['User_ID','Product_Category','Amount2']

Amount_CU3 = pd.DataFrame(full.groupby(['User_ID','Product_Category_3'])['Purchase'].sum())
Amount_CU3.reset_index(level=['User_ID','Product_Category_3'], inplace=True)
Amount_CU3.columns = ['User_ID','Product_Category','Amount3']

Amount_CU = Amount_CU1.merge(Amount_CU2, how="outer").merge(Amount_CU3, how="outer")
Amount_CU.fillna(0,inplace = True)
Amount_CU['Category_User_Amount'] = Amount_CU.Amount1 + Amount_CU.Amount2 + Amount_CU.Amount3
Amount_CU.drop(['Amount1','Amount2','Amount3'], axis = 1 ,inplace=True)
Amount_CU['Category_User_Rank'] = Amount_CU.groupby(['User_ID'])['Category_User_Amount'].rank(ascending=False)

# Cruzamos los dos anteriores para tener un ratio Usuario - Categoria

User_Category = Amount_CU.merge(Amount_byUser, how="left")
User_Category['Category_User_Ratio'] = (User_Category.Category_User_Amount / User_Category.User_Amount ) * 100
User_Category.head()

,User_ID,Product_Category,Category_User_Amount,Category_User_Rank,User_Amount,User_Rank,Category_User_Ratio
0,1000001,1,61604.0,4.0,334093.0,3801.0,18.439177
1,1000001,2,40027.0,7.0,334093.0,3801.0,11.980796
2,1000001,3,119859.0,3.0,334093.0,3801.0,35.875939
3,1000001,4,129943.0,1.0,334093.0,3801.0,38.894260
4,1000001,5,52068.0,6.0,334093.0,3801.0,15.584882


In [54]:
# Ranking/Participación Producto por Categoría

# 1. Obtenemos Monto por producto
Amount_byProduct = pd.DataFrame(full.groupby(['Product_ID'])['Purchase'].sum())
Amount_byProduct.columns = ['Product_Amount']
Amount_byProduct.reset_index(level=0, inplace=True)
Amount_byProduct['Product_Rank'] = Amount_byProduct['Product_Amount'].rank(ascending=False)

# 2. Obtenemos Monto por categoría
Amount_byC1 = pd.DataFrame(full.groupby(['Product_Category_1'])['Purchase'].sum())
Amount_byC1.reset_index(level=0, inplace=True)
Amount_byC1.columns = ['Product_Category','Amount1']

Amount_byC2 = pd.DataFrame(full.groupby(['Product_Category_2'])['Purchase'].sum())
Amount_byC2.reset_index(level=0, inplace=True)
Amount_byC2.columns = ['Product_Category','Amount2']

Amount_byC3 = pd.DataFrame(full.groupby(['Product_Category_3'])['Purchase'].sum())
Amount_byC3.reset_index(level=0, inplace=True)
Amount_byC3.columns = ['Product_Category','Amount3']

Amount_byCat = Amount_byC1.merge(Amount_byC2, how="outer").merge(Amount_byC3, how="outer")
Amount_byCat.fillna(0,inplace = True)
Amount_byCat['Category_Amount'] = Amount_byC1.Amount1 + Amount_byC2.Amount2 + Amount_byC3.Amount3
Amount_byCat.drop(['Amount1','Amount2','Amount3'], axis = 1 ,inplace=True)
Amount_byCat['Category_Rank'] = Amount_byCat['Category_Amount'].rank(ascending=False)


# 3. Obtenemos Relación categoría - producto
Product_Category_1 = full[['Product_ID','Product_Category_1']].drop_duplicates()
Product_Category_1.columns = ['Product_ID','Product_Category']

Product_Category_2 = full[['Product_ID','Product_Category_2']].drop_duplicates()
Product_Category_2 = Product_Category_2[~Product_Category_2.Product_Category_2.isnull()]
Product_Category_2.columns = ['Product_ID','Product_Category']

Product_Category_3 = full[['Product_ID','Product_Category_3']].drop_duplicates()
Product_Category_3 = Product_Category_3[~Product_Category_3.Product_Category_3.isnull()]
Product_Category_3.columns = ['Product_ID','Product_Category']

Product_Category = pd.concat([Product_Category_1, Product_Category_2,Product_Category_3], ignore_index=True)
Product_Category.drop_duplicates(inplace=True)

# 4. Juntamos los outputs anteriores y calculamos participacion y ranking
Product_Category = Product_Category.merge(Amount_byCat, how="left").merge(Amount_byProduct, how="left")
Product_Category['Product_Category_Rank'] = Product_Category.groupby(['Product_Category'])['Product_Amount'].rank(ascending=False)
Product_Category['Product_Category_Ratio'] = (Product_Category.Product_Amount / Product_Category.Category_Amount) * 100
Product_Category.head()

,Product_ID,Product_Category,Category_Amount,Category_Rank,Product_Amount,Product_Rank,Product_Category_Rank,Product_Category_Ratio
0,P00069042,3.0,6.682383e+08,6.0,2694686.0,543.0,38.0,0.403252
1,P00248942,1.0,2.588863e+09,1.0,9472642.0,72.0,46.0,0.365900
2,P00087842,12.0,4.529700e+08,9.0,126265.0,2658.0,85.0,0.027875
3,P00085442,12.0,4.529700e+08,9.0,496203.0,1722.0,54.0,0.109544
4,P00285442,8.0,9.175718e+08,3.0,1561631.0,881.0,299.0,0.170192


In [69]:
# Cruzamos con la información de usuario - categoria de User_Categoria
merge1_columns = ['User_ID','Product_Category','Category_User_Amount','Category_User_Rank','Category_User_Ratio']

full = full.merge(User_Category, how = "left", left_on = ['User_ID','Product_Category_1'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_User_Amount':'Category1_User_Amount', 'Category_User_Rank' : 'Category1_User_Rank', 'Category_User_Ratio' : 'Category1_User_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)

full = full.merge(User_Category[merge1_columns], how = "left", left_on = ['User_ID','Product_Category_2'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_User_Amount':'Category2_User_Amount', 'Category_User_Rank' : 'Category2_User_Rank', 'Category_User_Ratio' : 'Category2_User_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)

full = full.merge(User_Category[merge1_columns], how = "left", left_on = ['User_ID','Product_Category_3'], right_on = ['User_ID','Product_Category'])
full.rename(columns={'Category_User_Amount':'Category3_User_Amount', 'Category_User_Rank' : 'Category3_User_Rank', 'Category_User_Ratio' : 'Category3_User_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)

full.update(full[['Category3_User_Rank','Category2_User_Rank']].fillna(999))
full.update(full[['Category2_User_Amount','Category3_User_Amount','Category2_User_Ratio','Category3_User_Ratio']].fillna(0))

full.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,...,Category1_User_Rank,User_Amount,User_Rank,Category1_User_Ratio,Category2_User_Amount,Category2_User_Rank,Category2_User_Ratio,Category3_User_Amount,Category3_User_Rank,Category3_User_Ratio
0,0-17,A,F,0,10,3,NaN,NaN,P00069042,8370.0,...,3.0,334093.0,3801.0,35.875939,0.0,999.0,0.000000,0.0,999.0,0.000000
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,...,4.0,334093.0,3801.0,18.439177,31822.0,9.0,9.524893,27268.0,11.0,8.161799
2,0-17,A,F,0,10,12,NaN,NaN,P00087842,1422.0,...,5.0,334093.0,3801.0,18.341899,0.0,999.0,0.000000,0.0,999.0,0.000000
3,0-17,A,F,0,10,12,14.0,NaN,P00085442,1057.0,...,5.0,334093.0,3801.0,18.341899,27268.0,11.0,8.161799,0.0,999.0,0.000000
4,55+,C,M,0,16,8,NaN,NaN,P00285442,7969.0,...,1.0,810472.0,2084.0,56.775435,0.0,999.0,0.000000,0.0,999.0,0.000000


In [70]:
# Cruzamos con la información de producto - categoria de User_Categoria
merge2_columns = ['Product_ID','Product_Category','Category_Amount','Category_Rank','Product_Category_Rank', 'Product_Category_Ratio']

full = pd.merge(full, Product_Category, how="left", left_on = ['Product_ID','Product_Category_1'], right_on = ['Product_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Category1_Amount', 'Category_Rank': 'Category1_Rank', 'Product_Category_Rank' : 'Product_Category1_Rank', 'Product_Category_Ratio':'Product_Category1_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)
                
full = pd.merge(full, Product_Category[merge2_columns], how="left", left_on = ['Product_ID','Product_Category_2'], right_on = ['Product_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Category2_Amount', 'Category_Rank': 'Category2_Rank', 'Product_Category_Rank' : 'Product_Category2_Rank', 'Product_Category_Ratio':'Product_Category2_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)

full = pd.merge(full, Product_Category[merge2_columns], how="left", left_on = ['Product_ID','Product_Category_3'], right_on = ['Product_ID','Product_Category'])
full.rename(columns={'Category_Amount':'Category3_Amount', 'Category_Rank': 'Category3_Rank', 'Product_Category_Rank' : 'Product_Category3_Rank', 'Product_Category_Ratio':'Product_Category3_Ratio'}, inplace=True)
full.drop(['Product_Category'], axis = 1 ,inplace=True)

# Completamos los 0
full.update(full[['Category1_Amount','Product_Category1_Ratio']].fillna(0))
full.update(full[['Category2_Amount','Product_Category2_Ratio']].fillna(0))
full.update(full[['Category3_Amount','Product_Category3_Ratio']].fillna(0))
full.update(full[['Category1_Rank','Product_Category1_Rank','Category2_Rank','Product_Category2_Rank','Category3_Rank','Product_Category3_Rank']].fillna(999))
full.update(full[['Product_Category_1','Product_Category_2','Product_Category_3']].fillna(0))
full.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,...,Product_Category1_Rank,Product_Category1_Ratio,Category2_Amount,Category2_Rank,Product_Category2_Rank,Product_Category2_Ratio,Category3_Amount,Category3_Rank,Product_Category3_Rank,Product_Category3_Ratio
0,0-17,A,F,0,10,3,0.0,0.0,P00069042,8370.0,...,38.0,0.403252,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,...,46.0,0.365900,449248736.0,10.0,12.0,2.108552,608663460.0,7.0,3.0,1.556302
2,0-17,A,F,0,10,12,0.0,0.0,P00087842,1422.0,...,85.0,0.027875,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000
3,0-17,A,F,0,10,12,14.0,0.0,P00085442,1057.0,...,54.0,0.109544,608663460.0,7.0,227.0,0.081523,0.0,999.0,999.0,0.000000
4,55+,C,M,0,16,8,0.0,0.0,P00285442,7969.0,...,299.0,0.170192,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000


In [71]:
# mapeamos informacion
mapping_age = {'0-17': 1, '18-25': 2, '26-35': 3, '36-45' : 4, '46-50'  : 5, '51-55':6, '55+' : 7}
mapping_siccy = {'1': 1, '2': 2, '3': 3, '4+': 4, '0': 0}

# Transformamos strings
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
full.City_Category = le.fit_transform(full.City_Category)
full.Gender = le.fit_transform(full.Gender)
full.Product_ID = le.fit_transform(full.Product_ID)
full.User_ID = le.fit_transform(full.User_ID)
full.replace({'Age': mapping_age, 'Stay_In_Current_City_Years': mapping_siccy}, inplace = True)
full.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,...,Product_Category1_Rank,Product_Category1_Ratio,Category2_Amount,Category2_Rank,Product_Category2_Rank,Product_Category2_Ratio,Category3_Amount,Category3_Rank,Product_Category3_Rank,Product_Category3_Ratio
0,1,0,0,0,10,3,0.0,0.0,684,8370.0,...,38.0,0.403252,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000
1,1,0,0,0,10,1,6.0,14.0,2406,15200.0,...,46.0,0.365900,449248736.0,10.0,12.0,2.108552,608663460.0,7.0,3.0,1.556302
2,1,0,0,0,10,12,0.0,0.0,868,1422.0,...,85.0,0.027875,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000
3,1,0,0,0,10,12,14.0,0.0,844,1057.0,...,54.0,0.109544,608663460.0,7.0,227.0,0.081523,0.0,999.0,999.0,0.000000
4,7,2,1,0,16,8,0.0,0.0,2769,7969.0,...,299.0,0.170192,0.0,999.0,999.0,0.000000,0.0,999.0,999.0,0.000000


In [72]:
full.columns

Index(['Age', 'City_Category', 'Gender', 'Marital_Status', 'Occupation',
       'Product_Category_1', 'Product_Category_2', 'Product_Category_3',
       'Product_ID', 'Purchase', 'Stay_In_Current_City_Years', 'User_ID',
       'Category1_User_Amount', 'Category1_User_Rank', 'User_Amount',
       'User_Rank', 'Category1_User_Ratio', 'Category2_User_Amount',
       'Category2_User_Rank', 'Category2_User_Ratio', 'Category3_User_Amount',
       'Category3_User_Rank', 'Category3_User_Ratio', 'Category1_Amount',
       'Category1_Rank', 'Product_Amount', 'Product_Rank',
       'Product_Category1_Rank', 'Product_Category1_Ratio', 'Category2_Amount',
       'Category2_Rank', 'Product_Category2_Rank', 'Product_Category2_Ratio',
       'Category3_Amount', 'Category3_Rank', 'Product_Category3_Rank',
       'Product_Category3_Ratio'],
      dtype='object')

In [81]:
fields_user_demographic = ['City_Category','Gender','Marital_Status', 'Occupation', 'Stay_In_Current_City_Years']
fields_tr_cat1 = ['Category1_Rank','Category1_User_Rank','Category1_User_Ratio','Product_Category1_Rank', 'Product_Category1_Ratio']
fields_tr_cat2 = ['Category2_Rank','Category2_User_Rank','Category2_User_Ratio','Product_Category2_Rank', 'Product_Category2_Ratio']
fields_tr_cat3 = ['Category3_Rank','Category3_User_Rank','Category3_User_Ratio','Product_Category3_Rank', 'Product_Category3_Ratio']
fields_usr = ['User_Rank']
fields_prod = ['Product_Rank']
fields_pc = ['Product_Category_1','Product_Category_2','Product_Category_3']
fields_id = ['User_ID','Product_ID']
fields_amounts = ['Category1_Amount','Category2_Amount','Category3_Amount']

fields = fields_user_demographic + fields_tr_cat1 + fields_tr_cat3 + fields_usr + fields_pc + fields_id + fields_amounts

In [77]:
import os
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.2.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

def get_model(estimator, parameters, X_train, y_train, scoring):  
    model = GridSearchCV(estimator, param_grid=parameters, scoring=scoring)
    model.fit(X_train, y_train)
    return model.best_estimator_

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

clf1 = LinearRegression()
#clf2 = Lasso(alpha = 0.01)
clf3 = RandomForestRegressor()
clf4 = xgb.XGBRegressor()
clf5 = KNeighborsRegressor(n_neighbors = 3)
#clf6 = SVR()

n_train = full[~full.Purchase.isnull()]
n_test = full[full.Purchase.isnull()]

y_train = np.asarray(n_train.Purchase)
X_train = n_train[fields]


res_reglin,res_rf,res_xgb, res_lasso,res_kn, res_svc = [],[],[],[],[],[]

for i in range(9,11):
    #res_reglin = np.append(res_reglin,cross_val_score(clf1, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_lasso = np.append(res_lasso,cross_val_score(clf2, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_rf = np.append(res_rf,cross_val_score(clf3, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_xgb = np.append(res_xgb,cross_val_score(clf4, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    res_kn = np.append(res_kn,cross_val_score(clf5, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))
    #res_svc = np.append(res_kn,cross_val_score(clf6, X_train, y_train, cv=i, scoring='neg_mean_absolute_error'))

#print(pd.Series(-res_reglin).describe())
print(pd.Series(-res_lasso).describe())
print(pd.Series(-res_rf).describe())
print(pd.Series(-res_xgb).describe())
print(pd.Series(-res_kn).describe())
#print(pd.Series(-res_svc).describe())

C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increa

In [17]:
rf = RandomForestRegressor()
scoring = make_scorer(mean_absolute_error, greater_is_better=False)
parameters = {'max_features' : [10], 'max_depth' : [18], 'n_estimators' : [60]}
reg_rf = get_model(rf, parameters, X_train, y_train, scoring)
reg_rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=18,
           max_features=10, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
XGB = xgb.XGBRegressor(seed= 42)
scoring = make_scorer(mean_absolute_error, greater_is_better=False)
parameters = {'max_depth':np.arange(2,8), 'learning_rate': np.linspace(0.01,0.3,10),'reg_alpha':np.linspace(0.1,1.0,5), 'reg_lambda': np.linspace(1.0,3.0,5)}
reg_xgb = get_model(XGB, parameters, X_train, y_train, scoring)
reg_xgb

In [30]:
clf3 = RandomForestRegressor(max_features = 10, max_depth = 18, n_estimators = 60)
clf3.fit(X_train, y_train)
pred = clf3.predict(n_test.drop('Purchase',axis=1))

submission = pd.DataFrame({
        "User_ID": test.User_ID,
        "Product_ID": test.Product_ID,
        "Purchase": pred
    }, columns = ["User_ID","Product_ID","Purchase"])
submission.to_csv('prediction_solutionv1.csv', index=False)